# Files

In [ ]:
games_file = '/work/games.csv'
themes_file = '/work/themes.csv'
mechanics_file = '/work/mechanics.csv'
subcat_file = '/work/subcategories.csv'

# Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import StandardScaler
from gower import gower_matrix
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval
import pinecone
from getpass import getpass
from tqdm.auto import tqdm
import gradio as gr
from PIL import Image
import requests
from io import BytesIO

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Data

In [ ]:
games = pd.read_csv(games_file)
games = games.sort_values(by='BayesAvgRating', ascending=False)
themes = pd.read_csv(themes_file)
mechanics = pd.read_csv(mechanics_file)
subcategories = pd.read_csv(subcat_file)

# Files

In [ ]:
games.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
14509,174430,Gloomhaven,gloomhaven game euroinspired tactical combat...,2017,3.8699,8.74997,8.51488,1.63975,1,4,...,21926,21926,1,1,0,0,0,0,0,0
13702,161936,Pandemic Legacy: Season 1,pandemic legacy cooperative campaign game over...,2015,2.8336,8.59678,8.44451,1.56179,2,4,...,21926,21926,1,1,0,0,0,0,0,0
17329,224517,Brass: Birmingham,brass birmingham economic strategy game sequel...,2018,3.9058,8.66562,8.41573,1.24126,2,4,...,21926,21926,0,1,0,0,0,0,0,0
14059,167791,Terraforming Mars,s mankind begin terraform planet mar giant cor...,2016,3.2441,8.41879,8.27421,1.38764,1,5,...,21926,21926,0,1,0,0,0,0,0,0
17834,233078,Twilight Imperium: Fourth Edition,twilight imperium fourth edition game galactic...,2017,4.2497,8.67916,8.25955,1.60640,3,6,...,21926,21926,1,1,0,0,0,0,0,0


In [ ]:
themes.head()

,BGGId,Adventure,Fantasy,Fighting,Environmental,Medical,Economic,Industry / Manufacturing,Transportation,Science Fiction,...,Theme_Fashion,Theme_Geocaching,Theme_Ecology,Theme_Chernobyl,Theme_Photography,Theme_French Foreign Legion,Theme_Cruise ships,Theme_Apache Tribes,Theme_Rivers,Theme_Flags identification
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
mechanics.head()

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,...,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
subcategories.head()

,BGGId,Exploration,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic
0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0


# Columns and Descriptions

In [ ]:
list(games)

['BGGId',
 'Name',
 'Description',
 'YearPublished',
 'GameWeight',
 'AvgRating',
 'BayesAvgRating',
 'StdDev',
 'MinPlayers',
 'MaxPlayers',
 'ComAgeRec',
 'LanguageEase',
 'BestPlayers',
 'GoodPlayers',
 'NumOwned',
 'NumWant',
 'NumWish',
 'NumWeightVotes',
 'MfgPlaytime',
 'ComMinPlaytime',
 'ComMaxPlaytime',
 'MfgAgeRec',
 'NumUserRatings',
 'NumComments',
 'NumAlternates',
 'NumExpansions',
 'NumImplementations',
 'IsReimplementation',
 'Family',
 'Kickstarted',
 'ImagePath',
 'Rank:boardgame',
 'Rank:strategygames',
 'Rank:abstracts',
 'Rank:familygames',
 'Rank:thematic',
 'Rank:cgs',
 'Rank:wargames',
 'Rank:partygames',
 'Rank:childrensgames',
 'Cat:Thematic',
 'Cat:Strategy',
 'Cat:War',
 'Cat:Family',
 'Cat:CGS',
 'Cat:Abstract',
 'Cat:Party',
 'Cat:Childrens']

In [ ]:
print('games', games.shape,'themes', themes.shape, 'mechanics', mechanics.shape)

games (21925, 48) themes (21925, 218) mechanics (21925, 158)


# Create a function to transform columns

In [ ]:
# define the custom function to concatenate the categories
def concatenate_categories(row):
    categories = []
    for column_name, value in row.iteritems():
        if column_name.startswith('Cat:') and value == 1:
            categories.append(column_name.split(':')[1])
    return ', '.join(categories)

In [ ]:
# apply the custom function to create the new 'category' column
games['category'] = games.apply(concatenate_categories, axis=1)

In [ ]:
# Define a function that converts 0s and 1s to column names
def binary_to_colname(df,val, col_index):
    if val == 1:
        return df.iloc[:,1:].columns[col_index]
    else:
        return ""

In [ ]:
# Apply the function to each element of the DataFrame
themes['themes'] = themes.iloc[:,1:].apply(lambda row: ','.join(filter(None, [binary_to_colname(themes,val, col_index) for col_index, val in enumerate(row)])), axis=1)

KeyboardInterrupt: 

In [ ]:
# Apply the function to each element of the DataFrame
mechanics['mechanics'] = mechanics.iloc[:,1:].apply(lambda row: ','.join(filter(None, [binary_to_colname(mechanics,val, col_index) for col_index, val in enumerate(row)])), axis=1)

In [ ]:
# Apply the function to each element of the DataFrame
subcategories['subcategories'] = subcategories.iloc[:,1:].apply(lambda row: ','.join(filter(None, [binary_to_colname(subcategories,val, col_index) for col_index, val in enumerate(row)])), axis=1)

# Merge DataFrames

In [ ]:
game_theme_mechs = pd.merge(pd.merge (pd.merge(games, themes, how='inner', on='BGGId'), mechanics, how='inner', on='BGGId'), subcategories, how='inner', on='BGGId')
game_theme_mechs

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic,subcategories
0,174430,Gloomhaven,gloomhaven game euroinspired tactical combat...,2017,3.8699,8.74997,8.51488,1.63975,1,4,...,1,0,0,0,0,0,0,0,0,"Exploration,Miniatures"
1,161936,Pandemic Legacy: Season 1,pandemic legacy cooperative campaign game over...,2015,2.8336,8.59678,8.44451,1.56179,2,4,...,0,0,0,0,0,0,0,0,0,
2,224517,Brass: Birmingham,brass birmingham economic strategy game sequel...,2018,3.9058,8.66562,8.41573,1.24126,2,4,...,0,0,0,0,0,0,0,0,0,
3,167791,Terraforming Mars,s mankind begin terraform planet mar giant cor...,2016,3.2441,8.41879,8.27421,1.38764,1,5,...,0,1,0,0,0,0,0,0,0,Territory Building
4,233078,Twilight Imperium: Fourth Edition,twilight imperium fourth edition game galactic...,2017,4.2497,8.67916,8.25955,1.60640,3,6,...,0,0,0,0,0,0,0,0,0,Exploration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,16398,War,war play standard card deck special themed d...,0,1.0000,2.29913,4.01735,1.64464,2,2,...,0,0,1,0,0,0,0,0,0,Card Game
21921,7316,Bingo,classic party game player cover place card bas...,1530,1.0435,2.88375,3.96529,1.76389,2,99,...,0,0,0,0,0,0,0,0,0,
21922,5048,Candy Land,create eleanor abbott early s entertain child ...,1949,1.1012,3.18585,3.79929,1.72107,2,4,...,0,0,0,0,0,0,0,0,0,
21923,5432,Chutes and Ladders,traditional game ancient india bring uk comm...,-200,1.0195,2.86709,3.61566,1.64086,2,6,...,0,0,0,0,0,0,0,0,0,


## Drop columns and transform game_theme_mechs

In [ ]:
game_theme_mechs['Name_Year'] = game_theme_mechs['Name'] + " (" + game_theme_mechs['YearPublished'].astype(str) + ")"

In [ ]:
original_matrix = game_theme_mechs.copy()

In [ ]:
# The name of each game is set as the index of the dataframe.
game_theme_mechs.set_index("BGGId", inplace=True)

In [ ]:
mapping_dict = game_theme_mechs['Name_Year'].to_dict()

In [ ]:
mapping_dict

{174430: 'Gloomhaven (2017)',
 161936: 'Pandemic Legacy: Season 1 (2015)',
 224517: 'Brass: Birmingham (2018)',
 167791: 'Terraforming Mars (2016)',
 233078: 'Twilight Imperium: Fourth Edition (2017)',
 291457: 'Gloomhaven: Jaws of the Lion (2020)',
 220308: 'Gaia Project (2017)',
 187645: 'Star Wars: Rebellion (2016)',
 182028: 'Through the Ages: A New Story of Civilization (2015)',
 115746: 'War of the Ring: Second Edition (2012)',
 162886: 'Spirit Island (2017)',
 193738: 'Great Western Trail (2016)',
 12333: 'Twilight Struggle (2005)',
 169786: 'Scythe (2016)',
 84876: 'The Castles of Burgundy (2011)',
 173346: '7 Wonders Duel (2015)',
 167355: 'Nemesis (2018)',
 120677: 'Terra Mystica (2012)',
 124361: 'Concordia (2013)',
 28720: 'Brass: Lancashire (2007)',
 177736: 'A Feast for Odin (2016)',
 266192: 'Wingspan (2019)',
 316554: 'Dune: Imperium (2020)',
 205637: 'Arkham Horror: The Card Game (2016)',
 183394: 'Viticulture Essential Edition (2015)',
 164928: 'Orléans (2014)',
 2371

# Similarities

In [ ]:
game_theme_mechs=game_theme_mechs.drop(['Name','Name_Year','Description','Family','ImagePath','themes','mechanics','GoodPlayers','category', 'subcategories'], axis=1)

In [ ]:
# The dataframe is cleaned by dropping any rows that have missing values.
game_theme_mechs.dropna(inplace=True)

$$
scaler = \frac{x-u}{s}
$$

In [ ]:
# The values in the dataframe are standardized
scaler = StandardScaler()
matrix = scaler.fit_transform(game_theme_mechs.values)

In [ ]:
print(matrix)

[[  0.14339455   2.17500682   2.53279992 ...  -0.12742559  -0.17273578
   -0.08708864]
 [  0.13328219   0.90155092   2.35733965 ...  -0.12742559  -0.17273578
   -0.08708864]
 [  0.14845073   2.21912249   2.43618739 ...  -0.12742559  -0.17273578
   -0.08708864]
 ...
 [ -0.20042578  -1.22730656  -3.84021388 ...  -0.12742559  -0.17273578
   -0.08708864]
 [-11.06615949  -1.3277035   -4.20531418 ...  -0.12742559  -0.17273578
   -0.08708864]
 [-16.62795897  -1.14313042  -4.39875687 ...  -0.12742559  -0.17273578
   -0.08708864]]


In [ ]:
relavant_cols = ['Name_Year', 'YearPublished', 'mechanics', 'category', 'themes', 'MinPlayers', 'MaxPlayers']

In [ ]:
game_theme_mechs.shape

(14082, 426)

# Cosine Similarity

The cosine similarity between two vectors $\vec{a}$ and $\vec{b}$ can be calculated using the following formula:


$\text{similarity}(\vec{a}, \vec{b}) = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\| \|\vec{b}\|}
= \frac{\sum_{i=1}^n a_i b_i}{\sqrt{\sum_{i=1}^n a_i^2} \sqrt{\sum_{i=1}^n b_i^2}}$



Here, $\vec{a} \cdot \vec{b}$ represents the dot product of the two vectors, and $\|\vec{a}\|$ and $\|\vec{b}\|$ represent their respective magnitudes.

The resulting value ranges from -1 to 1, with values closer to 1 indicating a higher degree of similarity between the two vectors.


In [ ]:
# Calculate cosine similarity between the games
game_similarities = cosine_similarity(matrix)

In [ ]:
game_theme_mechs.shape

(14082, 426)

In [ ]:
game_similarities.shape

(14082, 14082)

## Cosine Table Creation

In [ ]:
def cosine_similar_index(game_name, n_similar=10):
    # Get the index of the game in the dataframe
    game_idx = game_theme_mechs.index.get_loc(game_name)

    # Get the cosine similarities of the game with all other games
    game_similarities_with_others = game_similarities[game_idx]

    # Sort the games by similarity and select the most similar games
    similar_games_idx = np.argsort(game_similarities_with_others)[::-1][1:n_similar+1]

    return similar_games_idx

In [ ]:
# Create an empty DataFrame to store the results
cosine_df = pd.DataFrame(index=game_theme_mechs.index, columns=[f"Similar Game {i+1}" for i in range(10)])

In [ ]:
# Iterate over each game in the original DataFrame
for game_name in list(game_theme_mechs.index):
    # Get the top 10 similar games for the current game
    similar_games_idx=cosine_similar_index(game_name, 10)
    
    cosine_games = []
    # Get the names of the similar games
    cosine_games = game_theme_mechs.iloc[similar_games_idx].index.tolist()

    # Store the similar game names in the results DataFrame
    cosine_df.loc[game_name] = cosine_games

In [ ]:
#Replacce the BGGIds with the corresponding names
cosine_df.index = cosine_df.index.map(mapping_dict)
# Iterate over the columns of the DataFrame
for column in cosine_df.columns:
    # Iterate over the rows of the DataFrame and replace the IDs with names
    for index, row in cosine_df.iterrows():
        cosine_df.at[index, column] = mapping_dict.get(row[column], row[column])

In [ ]:
cosine_df

,Similar Game 1,Similar Game 2,Similar Game 3,Similar Game 4,Similar Game 5,Similar Game 6,Similar Game 7,Similar Game 8,Similar Game 9,Similar Game 10
BGGId,,,,,,,,,,
Gloomhaven (2017),Gloomhaven: Jaws of the Lion (2020),Warmachine (2003),Pandemic Legacy: Season 1 (2015),Mage Knight Board Game (2011),Vampire: The Masquerade – Heritage (2020),Clank!: Legacy – Acquisitions Incorporated (2019),Spirit Island (2017),Aeon's End: Legacy (2019),Dragonfire (2017),Blood Rage (2015)
Pandemic Legacy: Season 1 (2015),Pandemic Legacy: Season 2 (2017),Risk Legacy (2011),Pandemic (2008),Pandemic Legacy: Season 0 (2020),Aeon's End: Legacy (2019),Dragonfire (2017),SeaFall (2016),Zombie Kidz Evolution (2018),Pandemic: Iberia (2016),The Rise of Queensdale (2018)
Brass: Birmingham (2018),Brass: Lancashire (2007),Arriala: Canal de Garonne (2010),Carnevale: Vicious Fighting Along the Canals o...,Venice (2021),Canal Grande (2002),Canal King Brugge (2019),Age of Industry (2010),Jaipur (2009),Wealth of Nations (2008),At the Gates of Loyang (2009)
Terraforming Mars (2016),7 Wonders (2010),Everdell (2018),Through the Ages: A New Story of Civilization ...,Stone Age (2008),Power Grid (2004),Wingspan (2019),The Castles of Burgundy (2011),Splendor (2014),Mage Knight Board Game (2011),Gaia Project (2017)
Twilight Imperium: Fourth Edition (2017),Inis (2016),Nexus Ops (2005),Small World Underground (2011),King of Tokyo: Dark Edition (2020),The Ares Project (2011),Scythe (2016),Mountain Goats (2020),Beyond the Sun (2020),Twilight Imperium: Third Edition (2005),Zeus on the Loose (2006)
...,...,...,...,...,...,...,...,...,...,...
War (0),Marbles (-3000),Balancing Moon (0),Jacks (0),Ninety-Nine (0),Fan Tan (0),Hedbanz for Kids (0),Spejle Æg (0),Big Pirate (0),Nim (0),Four Twenty-one (0)
Bingo (1530),Rolling Realms (2021),Würfel Bingo (2007),Super Mega Lucky Box (2021),Metro X (2018),Miscellaneous Game Compilation (0),Qwixx (2012),Super-Skill Pinball: 4-Cade (2020),Via Magica (2020),Rise of Augustus (2013),Take it Easy! (1983)
Candy Land (1949),The Hamburger Game (1989),Peanut Butter & Jelly Card Game (1971),Grzybobranie (1975),Pizza Party (1987),Hi Ho! Cherry-O (1960),Orchard (1986),Shopping List (1995),Pustekuchen (2003),The Grape Escape (1992),Slamwich (1994)


## New Function

In [ ]:
game = 'Terraforming Mars (2016)'

In [ ]:
similars_cosine = cosine_df.loc[game].tolist()
similars_cosine

['7 Wonders (2010)',
 'Everdell (2018)',
 'Through the Ages: A New Story of Civilization (2015)',
 'Stone Age (2008)',
 'Power Grid (2004)',
 'Wingspan (2019)',
 'The Castles of Burgundy (2011)',
 'Splendor (2014)',
 'Mage Knight Board Game (2011)',
 'Gaia Project (2017)']

In [ ]:
similars_cosine.insert(0,game)

In [ ]:
original_matrix[original_matrix['Name_Year'].isin(similars_cosine)][relavant_cols].sort_values(by=['Name_Year'], key=lambda x: x.map({v: i for i, v in enumerate(similars_cosine)}))


,Name_Year,YearPublished,mechanics,category,themes,MinPlayers,MaxPlayers
3,Terraforming Mars (2016),2016,"Hand Management,Hexagon Grid,Set Collection,Ti...",Strategy,"Environmental,Economic,Industry / Manufacturin...",1,5
74,7 Wonders (2010),2010,"Hand Management,Simultaneous Action Selection,...","Strategy, Family","Economic,Civilization,Ancient,City Building",2,7
28,Everdell (2018),2018,"Hand Management,Income,Variable Set-up,Solo / ...","Strategy, Family","Fantasy,Animals,City Building,Theme_Anthropomo...",1,4
8,Through the Ages: A New Story of Civilization ...,2015,"Auction/Bidding,Action Points,Income,Events,Ta...",Strategy,"Economic,Civilization",2,4
127,Stone Age (2008),2008,"Dice Rolling,Set Collection,End Game Bonuses,W...","Strategy, Family","Economic,Prehistoric",2,4
48,Power Grid (2004),2004,"Auction/Bidding,Market,Network and Route Build...",Strategy,"Economic,Industry / Manufacturing",2,6
21,Wingspan (2019),2019,"Dice Rolling,Hand Management,Set Collection,So...","Strategy, Family","Animals,Theme_Biology",1,5
14,The Castles of Burgundy (2011),2011,"Dice Rolling,Hexagon Grid,Set Collection,Tile ...",Strategy,Medieval,2,4
184,Splendor (2014),2014,"Set Collection,Race,Drafting,TableauBuilding,C...",Family,"Economic,Renaissance",2,4
27,Mage Knight Board Game (2011),2011,"Dice Rolling,Hand Management,Hexagon Grid,Modu...","Thematic, Strategy","Adventure,Fantasy,Fighting",1,4


# Gower Distance

Gower distance, is a distance metric used in data analysis to measure the similarity between objects, taking into account different types of variables (numeric, categorical, etc.). It is especially useful when working with mixed data sets, which contain different types of attributes.

In the context of a content-based board game recommendation system, Gower's distance can be beneficial in calculating the similarity between games. It can be used to compare the characteristics of the games, considering both numerical and categorical variables.

The mathematical formula to calculate Gower’s Distance:  $S_{ij}= \frac{\sum_{k}^{n} W_{ijk} S_{ijk}}{\sum_{k}^{n} W_{ijk}}$



## Original Matrix

In [ ]:
# Calculate the Gower distance matrix
gower_original_matrix = gower_matrix(original_matrix)


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
#Identify each game using the BGGId, since they follow the same order, we can use the series of the first DataFrame
gower_original = pd.DataFrame(gower_original_matrix, index=original_matrix['BGGId'])

In [ ]:
gower_original.columns = gower_original.index

In [ ]:
reduced_distances = pd.DataFrame(index=original_matrix.index, columns=[f"Similar Game {i+1}" for i in range(10)])

In [ ]:
for i in gower_original.index:
    input_game_distances = gower_original[gower_original.index == i]
    similar_game_indices = input_game_distances.iloc[0].sort_values().index[1:11]
    reduced_distances.loc[i] = similar_game_indices

In [ ]:
#Replacce the BGGIds with the corresponding names
reduced_distances.index = reduced_distances.index.map(mapping_dict)
# Iterate over the columns of the DataFrame
for column in reduced_distances.columns:
    # Iterate over the rows of the DataFrame and replace the IDs with names
    for index, row in reduced_distances.iterrows():
        reduced_distances.at[index, column] = mapping_dict.get(row[column], row[column])

## Game Theme Mechs

In [ ]:
distance_matrix_partial = gower_matrix(game_theme_mechs)

In [ ]:
distance_matrix_partial

In [ ]:
distance_matrix_partial.shape

(14082, 14082)

In [ ]:
gower_partial = pd.DataFrame(distance_matrix_partial, index=game_theme_mechs.index, columns=game_theme_mechs.index)
gower_partial

BGGId,174430,161936,224517,167791,233078,291457,220308,187645,182028,115746,...,3737,3522,2921,1406,1410,16398,7316,5048,5432,11901
BGGId,,,,,,,,,,,,,,,,,,,,,
174430,0.000000,0.068266,0.091984,0.085525,0.094067,0.019645,0.083195,0.080525,0.090678,0.083780,...,0.097271,0.096024,0.096983,0.122308,0.096248,0.090273,0.101134,0.096211,0.104806,0.102497
161936,0.068266,0.000000,0.060836,0.055943,0.066620,0.059417,0.061942,0.053940,0.049124,0.071480,...,0.058823,0.061745,0.062468,0.073978,0.062130,0.056268,0.067269,0.061800,0.075455,0.068586
224517,0.091984,0.060836,0.000000,0.061135,0.079908,0.087764,0.044829,0.070904,0.047588,0.079111,...,0.069228,0.067582,0.063733,0.075618,0.067805,0.061831,0.072818,0.067768,0.081298,0.074190
167791,0.085525,0.055943,0.061135,0.000000,0.078153,0.079626,0.037279,0.068193,0.045625,0.095778,...,0.074922,0.073951,0.069203,0.085612,0.073998,0.068136,0.079132,0.074005,0.087639,0.080445
233078,0.094067,0.066620,0.079908,0.078153,0.000000,0.087677,0.055535,0.063444,0.066669,0.088632,...,0.087761,0.080365,0.081853,0.094886,0.086255,0.080280,0.091258,0.085897,0.089801,0.092630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16398,0.090273,0.056268,0.061831,0.068136,0.080280,0.076099,0.061374,0.054066,0.048686,0.070179,...,0.012988,0.010008,0.023089,0.051791,0.009243,0.000000,0.023541,0.009821,0.021204,0.013699
7316,0.101134,0.067269,0.072818,0.079132,0.091258,0.087297,0.072361,0.060454,0.059673,0.081500,...,0.033299,0.020925,0.033920,0.062420,0.030776,0.023541,0.000000,0.030388,0.038056,0.035832
5048,0.096211,0.061800,0.067768,0.074005,0.085897,0.082399,0.067312,0.060366,0.054652,0.076478,...,0.018887,0.015903,0.023831,0.052420,0.006500,0.009821,0.030388,0.000000,0.023506,0.021930


In [ ]:
gower_partial.to_csv('/work/gower_partial.csv', index=True)

In [ ]:
gower_partial = pd.read_csv('gower_partial.csv', index_col=0)

In [ ]:
gower_partial

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# Input game name
input_game_name = 'Cards Against Humanity' 

# Search the game index name
input_game_index = original_matrix.index.get_loc(original_matrix[original_matrix['Name']==input_game_name].index[0])

In [ ]:
# Calculate the Gower distances between the game entered and the other games
input_game_distances = gower_original[input_game_index, :]

# Get the indices of the most similar games to the entered game
similar_game_indices = input_game_distances.argsort()[1:6]

# Get recommended similar games
similar_games = original_matrix.iloc[similar_game_indices]

NameError: name 'gower_original' is not defined

In [0]:
Similars=similar_games.index.tolist()

In [0]:
Similars.insert(0,input_game_name)
Similars

In [0]:
original_matrix[original_matrix['Name'].isin(Similars)][relavant_cols].sort_values(by=['Name'], key=lambda x: x.map({v: i for i, v in enumerate(Similars)}))

In [0]:
distances= sorted(input_game_distances, reverse=False)[1:6]
print(distances)

# NLP

In [ ]:
gower_distances_original = gower_distances_original.iloc[:,0:5]

In [ ]:
relavant_cols = ['Name_Year', 'YearPublished', 'mechanics', 'category','subcategories', 'themes', 'MinPlayers', 'MaxPlayers', 'ImagePath']

## List of text

In [ ]:
mechs_names = list(mechanics.columns)
if 'BGGId' in mechs_names:
    mechs_names.remove('BGGId')

theme_names = list(themes.columns)
if 'BGGId' in theme_names:
    theme_names.remove('BGGId')

subcategories_names = list(subcategories.columns)
if 'BGGId'in subcategories_names:
    subcategories_names.remove('BGGId')
    
categories_names = list(games['category'])
if 'BGGId'in categories_names:
    categories_names.remove('BGGId')

## Transform columns for analysis

In [ ]:
semantic=original_matrix

In [ ]:
def concatenar_lista(lista):
    lista = literal_eval(lista)
    return ' '.join(lista)

def concatenar_lista(lista):
    lista = literal_eval(lista)
    return ' '.join(lista)

In [ ]:
semantic = semantic.fillna(' ')

In [ ]:
semantic['text'] = semantic.apply(lambda x : str(x['Name'])+' '+x['Description']+' '+x['Family']+' '+x['category']+' '+x['subcategories']+' '+x['themes']+' '+x['mechanics'], axis=1)

In [ ]:
semantic.head(3)

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic,subcategories,Name_Year,text
0,174430,Gloomhaven,gloomhaven game euroinspired tactical combat...,2017,3.8699,8.74997,8.51488,1.63975,1,4,...,0,0,0,0,0,0,0,"Exploration,Miniatures",Gloomhaven (2017),Gloomhaven gloomhaven game euroinspired tact...
1,161936,Pandemic Legacy: Season 1,pandemic legacy cooperative campaign game over...,2015,2.8336,8.59678,8.44451,1.56179,2,4,...,0,0,0,0,0,0,0,,Pandemic Legacy: Season 1 (2015),Pandemic Legacy: Season 1 pandemic legacy coop...
2,224517,Brass: Birmingham,brass birmingham economic strategy game sequel...,2018,3.9058,8.66562,8.41573,1.24126,2,4,...,0,0,0,0,0,0,0,,Brass: Birmingham (2018),Brass: Birmingham brass birmingham economic st...


In [ ]:
Columns_text=['Name','YearPublished', 'Description', 'Family', 'category','subcategories', 'themes',
       'mechanics', 'ImagePath', 'text']

In [ ]:
semantic_analysis = semantic[Columns_text]

## Implementation of the NLP model

### Perplexity

In [ ]:
# Transform text columns into list
dataset=semantic_analysis['text'].tolist()

In [ ]:
def calculate_perplexity(model, dataset):
    """
    Calculate perplexity for a SentenceTransformer model and a data set.

    Args:
    - model: Previously loaded SentenceTransformer model
    - dataset: Data set as a list of sentences

    Returns:
    - perplexity: Calculated perplexity value
    """
    total_cosine_similarity = 0
    total_sentences = 0

    for i, sentence in enumerate(dataset):
        # Get the embedding for the sentence
        embedding = model.encode(sentence, convert_to_tensor=True)

        # Calculate the cosine similarity between the original sentence and the reconstructed sentence
        cosine_similarity = util.pytorch_cos_sim(embedding, model.encode(sentence, convert_to_tensor=True)).item()

        # Add cosine similarities
        total_cosine_similarity += cosine_similarity
        total_sentences += 1

    # Calculates the perplexity as the inverse of the average cosine similarity
    perplexity = 1 / (total_cosine_similarity / total_sentences)
    return perplexity

In [ ]:
# Import different proposed models
model_1 = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
model_2 = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Calculate the perplexity
Perplexity_RoBERTa=calculate_perplexity(model_1,dataset)
Perplexity_MiniLM=calculate_perplexity(model_2,dataset)

Print(Perplexity_RoBERTa)
Print(Perplexity_MiniLM)

De acuerdo al resultado obtenido, donde se observa que la perplejidad entre los dos modelos propuestos es muy similar; decidimos elegir el modelo que es menos costoso computacionalmente, por lo que se elige el modelo RoBERTa, para llevar a cabo el desarrollo e implementación de nuestro sistema de recomendaciones de juegos de mesa.

Use transformers and load the RoBERTa model

In [ ]:
model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 245kB/s]
README.md: 100%|██████████| 4.00k/4.00k [00:00<00:00, 5.16MB/s]
added_tokens.json: 100%|██████████| 2.00/2.00 [00:00<00:00, 2.70kB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 167kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 48.9MB/s]
pytorch_model.bin: 100%|██████████| 499M/499M [00:08<00:00, 57.3MB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 72.8kB/s]
sentence_roberta_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 76.4kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 342kB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 28.6MB/s]
tokenizer_config.json: 100%|██████████| 334/334 [00:00<00:00, 448kB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 56.1MB/s]
modules.json: 100%|██████████| 229/229 [00:00<00:00, 325kB/s]


In [ ]:
embeddings = model.encode(semantic_analysis['text'],batch_size=64,show_progress_bar=True)

Batches:  37%|███▋      | 128/343 [51:30<1:26:30, 24.14s/it]


KeyboardInterrupt: 

In [ ]:
semantic_analysis['embeddings'] = embeddings.tolist()
semantic_analysis['ids'] = df.index
semantic_analysis['ids'] = df['ids'].astype('str')

Use Pincone for save embeddings 

In [ ]:
pincone_api = getpass('Enter the secret value: ')

In [ ]:
pinecone.init(api_key=pincone_api, environment="gcp-starter")

In [ ]:
dimensions_embeddings = len(semantic_analysis['embeddings'][0])
index_name = 'games-embeddings'
all_index = pinecone.list_indexes()
if index_name in all_index:
    index = pinecone.Index(index_name)
else:
    pinecone.create_index(index_name, dimension=dimensions_embeddings, metric="cosine")

In [ ]:
index = pinecone.Index('games-embeddings')

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(semantic_analysis), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(semantic_analysis))
    # extract batch
    batch = semantic_analysis[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

Generate a test to observe the functionality of the model

In [ ]:
query = 'War dragons and fantasy'
query_vector = model.encode(query).tolist()

responses = index.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
)

In [ ]:
responses

{'matches': [{'id': '16713',
              'metadata': {'AvgRating': 563.562,
                           'BGGId': 11187.0,
                           'BayesAvgRating': 550.885,
                           'BestPlayers': 0.0,
                           'Description': 'create fantasy army counter '
                                          'miniature fight battle play edition '
                                          'adampd rpg game',
                           'Family': ' ',
                           'GameWeight': 29.394,
                           'ImagePath': 'https://cf.geekdo-images.com/knELdpTBFSUcl_bWtcZvjQ__original/img/OPtT0d_qeEbc_QhXuGDzyNKAntY=/0x0/filters:format(jpeg)/pic167244.jpg',
                           'MaxPlayers': 2.0,
                           'MinPlayers': 2.0,
                           'Name': 'Advanced Dungeons & Dragons Battlesystem',
                           'Name_Year': 'Advanced Dungeons & Dragons '
                                        'Battlesyst

In [ ]:
!pip install pillow requests deepface gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 124.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 48.9 MB/s eta 0:00:00
  Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
  Created wheel f

In [ ]:
# Created a function to calculate similar games from the Gower distance
def search_game(game,top_k):

    # Get the list of Gower distances for the given game
    similars_gower = gower_distances_original.loc[game].tolist()
    similars_gower.insert(0, game)

    # Filter the original matrix based on Gower distances and conditions
    resultados = original_matrix[original_matrix['Name_Year'].isin(similars_gower)][relavant_cols]

    # Sort results according to the order of the Gower distance list

    resultados = resultados.sort_values(by=['Name_Year'], key=lambda x: x.map({v: i for i, v in enumerate(similars_gower)}))

    # Format the responses for better display
    response_data = []
    images = []  

    for _, row in resultados.iterrows():
        image_url = row['ImagePath']

        # Load the image from URL
        image_response = requests.get(image_url)
        image = Image.open(BytesIO(image_response.content))

        response_data.append({
            'Title': row['Name_Year'],
            'Year': row['YearPublished'],
            'Category': row['category'],
        })

        images.append(image)  

    df = pd.DataFrame(response_data)
    return df, images  

In [ ]:
# Created a function that returns recommended games based on semantic search
def search(query, category,theme, top_k):

    # Convert to lowercase
    query_lower = query.lower()

    # Eliminar signos de puntuación
    query_no_punct = query_lower.translate(str.maketrans("", "", string.punctuation))

    # Remove punctuation marks
    words = word_tokenize(query_no_punct)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Join the processed words into a string
    processed_query = ' '.join(filtered_words)
    query_vector = model.encode(processed_query).tolist()

    if category:
        filter_category = category
    else:
        filter_category = ''

    if category:
         conditions ={
                "Category": { "$in": [subcategories] }}
    else:
        conditions ={
                "Category": { "$gte": filter_category },
                }

    responses = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,

    )

    # Format the responses for better display
    response_data = []
    images = []

    for response in responses['matches']:
        metadata = response['metadata']
        image_url = metadata['ImagePath']

        # Load the image from URL
        image_response = requests.get(image_url)
        image = Image.open(BytesIO(image_response.content))

        response_data.append({
            'Title': metadata['Name'],
            ##'Overview': metadata['Description'],
            'Year': metadata['YearPublished'],
            'Category': metadata['subcategories'],
            'Similarity': response['score']
        })

        images.append(image)

    df = pd.DataFrame(response_data)
    return df, images

### Implement a graphical interface from gradio

In [ ]:
category = subcategories_names
themes=theme_names
iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=5, placeholder="Escribe aquí tu descripción...", label="Consulta"),
        gr.Dropdown(choices=category, label="Categoria del juego de mesa"),
        gr.Dropdown(choices=themes, label="Tema del juego de mesa"),
        gr.Number(minimum=1, maximum=10, value=5, label="Número de resultados")
    ],
    outputs=[
        gr.Dataframe(type="pandas", label="Resultados"),
        gr.Gallery(label="Imágenes"),
    ],
    title="Buscador de Juegos de Mesa",
    description="Introduce una descripción del juego para consultar,",
)
name_games=list(original_matrix['Name_Year'])
similar = gr.Interface(
    fn=search_game,
    inputs=[
        gr.Dropdown(choices=name_games, label="Seleccione un juego de mesa"),
        ##gr.Number(minimum=1, maximum=10, value=5, label="Número de resultados")
    ],
    outputs=[
        gr.Dataframe(type="pandas", label="Resultados"),
        gr.Gallery(label="Imágenes"),  # Quita el argumento 'type'
    ],
    title="Buscador de Juegos de Mesa",
    description="Introduce una juego para consultar,",
)

# Launch the interface
demo = gr.TabbedInterface([iface, similar], ["Semantic Search", "Similar Games"])

if __name__ == "__main__":
    demo.launch()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a6ce863a-f8e0-494d-8cc1-3239a140e9e5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>